In [ ]:
%%capture
import pandas as pd
from django_pandas.io import read_frame
from pathlib import Path
from dj_notebook import activate

plus = activate(dotenv_file="/Users/erikvw/source/edc_source/meta-edc/.env")
report_folder = Path("/Users/erikvw/Documents/ucl/protocols/meta3/reports/")
# output is suppressed -- normally would spew out all the edc loading messages


In [ ]:
from meta_rando.models import RandomizationList
from meta_consent.models import SubjectConsent


In [ ]:
cnt = 0
df_hm = pd.read_excel("/Users/erikvw/Documents/ucl/protocols/meta3/pharmacy/HINDU_MANDAL.xlsx")
df_hm = df_hm.groupby("sid").agg({"qty":"sum"})
cnt += len(df_hm)
print("df_hm", len(df_hm), cnt)

df_amana = pd.read_excel("/Users/erikvw/Documents/ucl/protocols/meta3/pharmacy/AMANA.xlsx")
df_amana = df_amana.groupby("sid").agg({"qty":"sum"})
cnt += len(df_amana)
print("df_amana", len(df_amana), cnt)

df_mnazi = pd.read_excel("/Users/erikvw/Documents/ucl/protocols/meta3/pharmacy/mnazi.xlsx")
df_mnazi = df_mnazi.groupby("sid").agg({"qty":"sum"})
cnt += len(df_mnazi)
print("df_mnazi", len(df_mnazi), cnt)

df_mwana = pd.read_excel("/Users/erikvw/Documents/ucl/protocols/meta3/pharmacy/MWANANYAMALA.xlsx")
df_mwana = df_mwana.groupby("sid").agg({"qty":"sum"})
cnt += len(df_mwana)
print("df_mwana", len(df_mwana), cnt)

df_temeke = pd.read_excel("/Users/erikvw/Documents/ucl/protocols/meta3/pharmacy/TEMEKE.xlsx")
df_temeke = df_temeke.groupby("sid").agg({"qty":"sum"})
cnt += len(df_temeke)
print("df_temeke", len(df_temeke), cnt)

df_sites = pd.concat([df_hm, df_amana, df_mnazi, df_mwana, df_temeke], axis=0)
df_sites

In [ ]:
# get rando, this is slow (col assignment is encrypted)
df_rando = read_frame(RandomizationList.objects.values("subject_identifier", "sid", "site_name", "assignment", "allocated_site").all(), verbose=False)
df_rando = df_rando.rename(columns={"allocated_site": "site_id"})
df_rando = df_rando[["subject_identifier", "sid", "site_name", "assignment", "site_id"]]
df_rando = df_rando.reset_index(drop=True)


In [ ]:
# merge rando w/ hindu_mandal
df_sites = df_sites.merge(df_rando, on='sid', how='left', suffixes=('', '_rando'))
df_sites = df_sites[["subject_identifier", "sid", "site_id", "qty", "site_name", "assignment"]]
df_sites


In [ ]:
# merge hindu_mandal w/ consent to get "consent_datetime", "gender", "dob"
df_consent = read_frame(SubjectConsent.objects.values("subject_identifier", "consent_datetime", "gender", "dob"), verbose=False)
df_consent["consent_datetime"] = pd.to_datetime(df_consent["consent_datetime"]).dt.tz_localize(None)
df_consent["consent_datetime"] = df_consent["consent_datetime"].dt.normalize()
df_sites = df_sites.merge(df_consent, on="subject_identifier", how="left")
df_sites

In [ ]:
df_sites = df_sites.reset_index(drop=True)
df_sites


In [ ]:
df_sites.site_name.value_counts()

In [ ]:
df_sites

In [ ]:
df_sites[(df_sites.qty.notna())]["site_name"].value_counts()

In [ ]:
df_sites[(df_sites.subject_identifier.isna())]["site_name"].value_counts()


In [ ]:
# lists SIDs that do not match an allocated subject_identifier 
export_errors_df = df_sites[(df_sites.subject_identifier.isna())][["subject_identifier", "sid", "site_id", "qty", "site_name", "consent_datetime", "gender", "dob"]].copy()
export_errors_df = export_errors_df.reset_index(drop=True)
export_errors_df.to_csv("/Users/erikvw/Documents/ucl/protocols/meta3/pharmacy/imp_stock_20241016-errors.csv")


In [ ]:
# safe file to show pharmacist and coordinators
export_wo_assignment_df = df_sites[(df_sites.subject_identifier.notna()) & (df_sites.qty.notna())][["subject_identifier", "sid", "site_id", "qty", "site_name", "consent_datetime", "gender", "dob"]].copy()
export_wo_assignment_df = export_wo_assignment_df.reset_index(drop=True)
export_wo_assignment_df.to_csv("/Users/erikvw/Documents/ucl/protocols/meta3/pharmacy/imp_stock_20241016-calvin.csv")

In [ ]:
# this is a secure file, for Buma only!!
export_with_assignment_df = df_sites[(df_sites.subject_identifier.notna()) & (df_sites.qty.notna())][["subject_identifier", "sid", "site_id", "qty", "site_name", "consent_datetime", "gender", "dob", "assignment"]].copy()
export_with_assignment_df = export_with_assignment_df.reset_index(drop=True)
export_with_assignment_df.to_csv("/Users/erikvw/Documents/ucl/protocols/meta3/pharmacy/imp_stock_20241016-buma.csv")


In [ ]:
# summary of bottles
btl_df = export_wo_assignment_df.groupby("site_name").agg({"qty":"sum"})
btl_df

In [ ]:
# summary of subjects
export_wo_assignment_df.site_name.value_counts()

In [ ]:
# summary of tabs
btl_df * 128

In [ ]:
(btl_df * 128).qty.sum()

In [ ]:
df_assign = export_with_assignment_df.groupby("assignment").agg({"qty":"sum"})
df_assign

In [ ]:
df_assign * 128

In [ ]:
(df_assign * 128).qty.sum()